<a href="https://colab.research.google.com/github/mudogruer/LLMs/blob/main/gemma_7b_zero_shot_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U accelerate bitsandbytes bert-score evaluate sentencepiece

In [ ]:
!pip install -q -U transformers

installed the necessary libraries

In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quanty_type = "nf4",
    bnb_4bit_use_double_quanty = True,
)


Making configurations to load data in 4 bits

In [ ]:
from huggingface_hub import login
login()

We need to login to huggingface to reach gemma model

In [ ]:
model_name = "google/gemma-7b-it"

from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

Uploaded gemma model tokenizer

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code = True,

)

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Loaded gemma model

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    device_map = "auto"

)


2024-02-22 15:26:48.813426: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 15:26:48.813525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 15:26:48.987115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


created a pipeline to use the model

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
prompt = "<start_of_turn>user"+"What is the heaviest element?"+"<end_of_turn><start_of_turn>model"

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)

making some basic experiment

In [ ]:
print(sequence[0]['generated_text'])

<start_of_turn>userWhat is the heaviest element?<end_of_turn><start_of_turn>modelThe heaviest element is is element alloy. alloy. element alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy


In [ ]:
sequence

[{'generated_text': '<start_of_turn>userWhat is the heaviest element?<end_of_turn><start_of_turn>modelThe heaviest element is is element alloy. alloy. element alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy alloy'}]

In [ ]:
def Sequence(promt):
    sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 30,
    temperature = 0.1,
    top_k = 2,
    top_p = 1.0,
    pad_token_id=tokenizer.eos_token_id
    )
    return sequence

Create a function to inference from multi questions

In [ ]:
prompt = "<start_of_turn>user"+"Who invented mirror telescope?"+"<end_of_turn><start_of_turn>model"

print(Sequence(prompt)[0]['generated_text'])

<start_of_turn>userWho invented mirror telescope?<end_of_turn><start_of_turn>model invent Mirror telescope Invention Invention

Mirror telescope invention invention occurred in 17th century by a inventor named named named William. The first prototype prototype was


Checking our format

In [ ]:
from datasets import load_dataset

dataset = load_dataset("sciq")

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset sciq downloaded and prepared to /root/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Loaded a dataset which includes scientific question-answers

In [ ]:
dataset

Checking dataset format

In [ ]:
dataset_validation = load_dataset('sciq',split='validation[:20%]') #first %20 of validation data

Created a dataset using %20 of validation dataset

In [ ]:
dataset_validation

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 200
})

In [ ]:
answers = []
for question in dataset_validation["question"]:
    prompt = "<start_of_turn>user:"+question+"\n<end_of_turn><start_of_turn>model:"
    output_sequences = Sequence(prompt)[0]['generated_text']
    question_and_answer = output_sequences.split("\n<end_of_turn><start_of_turn>model")
    answer = question_and_answer[1]
    answers.append(answer)

Making inferences for 200 questions and adding to a list named answers

In [ ]:
import pandas as pd
df = pd.DataFrame({"generated_answers":answers,"correct_answers":dataset_validation["correct_answer"]})

In [ ]:
df["generated_answers"][1]

':Each specific polypeptide has a unique linear sequence of amino acids that defines its unique structure. The unique sequence of amino acids allows each polypeptide to fold into its unique conformation.'

In [ ]:
df["correct_answers"][1]

'amino'

In [ ]:
from evaluate import load
import numpy as np
bertscore = load("bertscore")
predictions = answers
references = dataset_validation["correct_answer"]
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print("precision: ",round(np.mean(list(results["precision"])),5))
print("recall: ",round(np.mean(list(results["recall"])),5))
print("f1: ",round(np.mean(list(results["f1"])),5))

precision:  0.63073
recall:  0.71357
f1:  0.66882


We need to control of the quality of the answers. I used bert model to analyize answers which is better for generative use-cases.

# Personal notes:
Gemma 7b model is making really slow inferences. Nevertheless it still has lower scores than similar models.